In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from wordcloud import WordCloud

import nltk
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence #unique id

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout, Embedding
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df=pd.read_csv("file.csv")

In [ ]:
df

,Unnamed: 0,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad
...,...,...,...
219289,219289,Other Software Projects Are Now Trying to Repl...,bad
219290,219290,I asked #ChatGPT to write a #NYE Joke for SEOs...,good
219291,219291,chatgpt is being disassembled until it can onl...,bad
219292,219292,2023 predictions by #chatGPT. Nothing really s...,bad


In [ ]:
df.isnull().sum()

Unnamed: 0    0
tweets        0
labels        0
dtype: int64

In [ ]:
df.labels.value_counts()

bad        107796
good        56011
neutral     55487
Name: labels, dtype: int64

In [ ]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [ ]:
df["tweets"]=df["tweets"].map(remove_emojis)

In [ ]:
def cleantext(text):
  tokens = word_tokenize(text.lower())
  ftoken = [t for t in tokens if(t.isalpha())]
  stop = stopwords.words("english")
  ctoken = [t for t in ftoken if(t not in stop)]
  lemma = WordNetLemmatizer()
  ltoken = [lemma.lemmatize(t) for t in ctoken]
  return " ".join(ltoken)

In [ ]:
df["tweets"]=df["tweets"].map(cleantext)

In [ ]:
df["labels"]=df["labels"].replace("neutral","good")

In [ ]:
df.labels.value_counts()

good    111498
bad     107796
Name: labels, dtype: int64

In [ ]:
x=df['tweets']
y=df["labels"]

In [ ]:
y=np.where(y=="good",1,0)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=1)


In [ ]:

sentlen = []
for sent in df["tweets"]:
  sentlen.append(len(word_tokenize(sent)))

df["SentLen"] = sentlen
df.head()


,Unnamed: 0,tweets,labels,SentLen
0,0,chatgpt optimizing language model dialogue htt...,good,7
1,1,try talking chatgpt new ai system optimized di...,good,13
2,2,chatgpt optimizing language model dialogue htt...,good,20
3,3,thrilled share chatgpt new model optimized dia...,good,14
4,4,minute ago openai released new chatgpt use rig...,bad,10


In [ ]:
max(sentlen)

46

In [ ]:

np.quantile(sentlen, 0.95)

25.0

In [ ]:

max_len = np.quantile(sentlen, 0.95)


In [ ]:
tok = Tokenizer(char_level=False, split=" ")
#char_level	if True, every character will be treated as a token.

tok.fit_on_texts(xtrain)
tok.index_word

{1: 'chatgpt',
 2: 'http',
 3: 'ai',
 4: 'openai',
 5: 'like',
 6: 'write',
 7: 'google',
 8: 'new',
 9: 'asked',
 10: 'using',
 11: 'use',
 12: 'answer',
 13: 'question',
 14: 'make',
 15: 'get',
 16: 'chatbot',
 17: 'code',
 18: 'time',
 19: 'know',
 20: 'good',
 21: 'thing',
 22: 'one',
 23: 'think',
 24: 'people',
 25: 'could',
 26: 'see',
 27: 'amp',
 28: 'via',
 29: 'way',
 30: 'human',
 31: 'search',
 32: 'work',
 33: 'would',
 34: 'tool',
 35: 'need',
 36: 'going',
 37: 'model',
 38: 'future',
 39: 'day',
 40: 'ask',
 41: 'world',
 42: 'bot',
 43: 'really',
 44: 'writing',
 45: 'even',
 46: 'year',
 47: 'better',
 48: 'language',
 49: 'technology',
 50: 'prompt',
 51: 'help',
 52: 'used',
 53: 'say',
 54: 'job',
 55: 'first',
 56: 'tech',
 57: 'story',
 58: 'artificialintelligence',
 59: 'chat',
 60: 'take',
 61: 'content',
 62: 'much',
 63: 'u',
 64: 'got',
 65: 'intelligence',
 66: 'give',
 67: 'still',
 68: 'twitter',
 69: 'create',
 70: 'response',
 71: 'article',
 72: 'als

In [ ]:

vocab_len = len(tok.index_word)
vocab_len


61626

In [ ]:

seqtrain = tok.texts_to_sequences(xtrain) #step1
seqtrain

[[458, 200, 2156, 176, 1, 241, 3314, 21, 152, 1493],
 [306, 2086, 29, 2040, 394, 239, 259, 1097, 9818, 2854, 1, 2],
 [49,
  38,
  434,
  294,
  103,
  101,
  48,
  37,
  77,
  416,
  376,
  48,
  729,
  268,
  1,
  411,
  617,
  1427],
 [115, 1916, 1, 2],
 [2126, 282, 158, 1883, 46, 8, 1, 91, 1, 4, 2126, 2],
 [8820, 140, 126, 118, 91, 1, 34, 1707, 316, 48, 1903, 417, 1, 28826, 58, 2],
 [148,
  332,
  134,
  87,
  16,
  325,
  543,
  508,
  376,
  48,
  554,
  523,
  30,
  584,
  105,
  965,
  81,
  1708,
  196,
  214,
  20969],
 [3587, 9, 1, 6, 621, 232, 1087, 2],
 [328, 94, 1, 14, 3162, 622, 297, 3486, 23, 94, 2],
 [159, 274, 1],
 [1, 4097, 2],
 [1, 133, 56, 1319, 395, 1, 2],
 [141, 310, 1, 51, 86, 1480, 81, 141, 3, 3, 1, 28827, 1471, 2],
 [1, 5, 189, 9819, 2414, 10432, 699, 444, 431, 4691, 272, 420, 63, 2],
 [22,
  425,
  78,
  28828,
  4591,
  9,
  4,
  3760,
  13238,
  17218,
  1277,
  83,
  1,
  295,
  27,
  55,
  3163,
  12046,
  3,
  4504,
  28829,
  5893,
  1795,
  2],
 [13, 4,

In [ ]:

seqmattrain = sequence.pad_sequences(seqtrain, maxlen= int(max_len)) #step2
seqmattrain

array([[   0,    0,    0, ...,   21,  152, 1493],
       [   0,    0,    0, ..., 2854,    1,    2],
       [   0,    0,    0, ...,  411,  617, 1427],
       ...,
       [   0,    0,    0, ...,  467,  383, 1694],
       [   0,    0,    0, ..., 3361, 1949,    2],
       [   0,    0,    0, ..., 3858, 4156,    2]], dtype=int32)

In [ ]:

seqtest = tok.texts_to_sequences(xtest)
seqmattest = sequence.pad_sequences(seqtest, maxlen=int(max_len))

In [ ]:


vocab_len

61626

In [ ]:

rnn = Sequential()

rnn.add(Embedding(vocab_len+1,70, input_length=int(max_len), mask_zero=True))
rnn.add(SimpleRNN(units=32, activation="tanh"))
rnn.add(Dense(units=32, activation="relu"))
rnn.add(Dropout(0.2))

rnn.add(Dense(units=1, activation="sigmoid"))

rnn.compile(optimizer="adam", loss="binary_crossentropy")

rnn.fit(seqmattrain, ytrain, batch_size=50, epochs=25)
ypred = rnn.predict(seqmattest)


Epoch 1/25
3071/3071 [==============================] - 221s 71ms/step - loss: 0.2835
Epoch 2/25
3071/3071 [==============================] - 216s 70ms/step - loss: 0.1606
Epoch 3/25
3071/3071 [==============================] - 210s 69ms/step - loss: 0.1078
Epoch 4/25
3071/3071 [==============================] - 210s 68ms/step - loss: 0.0706
Epoch 5/25
3071/3071 [==============================] - 209s 68ms/step - loss: 0.0519
Epoch 6/25
3071/3071 [==============================] - 213s 69ms/step - loss: 0.0410
Epoch 7/25
3071/3071 [==============================] - 212s 69ms/step - loss: 0.0336
Epoch 8/25
3071/3071 [==============================] - 212s 69ms/step - loss: 0.0287
Epoch 9/25
3071/3071 [==============================] - 209s 68ms/step - loss: 0.0268
Epoch 10/25
3071/3071 [==============================] - 200s 65ms/step - loss: 0.0239
Epoch 11/25
3071/3071 [==============================] - 200s 65ms/step - loss: 0.0215
Epoch 12/25
3071/3071 [=============================

In [ ]:
ypred= ypred>0.5
ypred=np.where(ypred==True,1,0)


In [ ]:
ypred

array([[1],
       [1],
       [1],
       ...,
       [0],
       [1],
       [0]])

In [ ]:
from sklearn.metrics import classification_report,accuracy_score


In [ ]:
print(classification_report(ytest,ypred))


              precision    recall  f1-score   support

           0       0.91      0.88      0.90     32288
           1       0.89      0.92      0.90     33501

    accuracy                           0.90     65789
   macro avg       0.90      0.90      0.90     65789
weighted avg       0.90      0.90      0.90     65789

